In [1]:
# Parameters 

application = 'Apple_and_Pears'
varname = 'RAIN_BC'
target_type = 'cat3'
GCM = 'ECMWF'
var_X = 'PRECIP'

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
%matplotlib inline

In [5]:
import sys 
import pathlib

In [6]:
import matplotlib.pyplot as plt 

In [7]:
import numpy as np 
import pandas as pd 

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

In [10]:
import autogluon as ag
from autogluon import TabularPrediction as task

In [11]:
np.random.seed(42)

In [12]:
HOME = pathlib.Path.home()
CWD = pathlib.Path.cwd()

In [13]:
sys.path.append('/home/nicolasf/research/Smart_Ideas/code/ml4seas')

### read the concatenated target and GCM data 

In [14]:
ipath = HOME.joinpath(f"research/Smart_Ideas/outputs/targets/application_cases/{application}/Join_GCM_targets") 

In [15]:
[x.name for x in list(ipath.glob("*.csv"))]

['CSV_Apple_and_Pears_TMEAN_cat3_ECMWF_T2M.csv',
 'CSV_Apple_and_Pears_RAIN_BC_cat3_ECMWF_PRECIP.csv',
 'CSV_Apple_and_Pears_TMEAN_cat5_ECMWF_T2M.csv',
 'CSV_Apple_and_Pears_RAIN_BC_cat3_ECMWF_Z500.csv']

In [16]:
data = pd.read_csv(ipath.joinpath(f"CSV_Apple_and_Pears_{varname}_cat3_{GCM}_{var_X}.csv"), index_col=0, parse_dates=True)

In [17]:
data.head()

,"(-70.0, 70.0)","(-70.0, 72.5)","(-70.0, 75.0)","(-70.0, 77.5)","(-70.0, 80.0)","(-70.0, 82.5)","(-70.0, 85.0)","(-70.0, 87.5)","(-70.0, 90.0)","(-70.0, 92.5)",...,"(60.0, 280.0)","(60.0, 282.5)","(60.0, 285.0)","(60.0, 287.5)","(60.0, 290.0)","(60.0, 292.5)","(60.0, 295.0)","(60.0, 297.5)","(60.0, 300.0)",cat3
instance,,,,,,,,,,,,,,,,,,,,,
1981-03-31,-12.950461,-10.646537,-9.865860,-9.913054,-7.438723,-7.315302,-7.397714,-5.950508,-4.236245,-3.099089,...,-1.017812,-0.047586,1.223597,1.856052,5.252983,3.054986,-0.573380,3.879854,-4.569364,2
1981-03-31,-10.627958,-15.364811,-20.773027,-16.256193,-11.049069,-8.784796,-7.914088,-7.414926,-5.939186,-4.268093,...,-1.402276,0.143256,4.262129,7.261390,12.535162,3.537306,6.287004,11.003606,10.323011,2
1981-03-31,9.447995,8.591861,9.746018,1.502732,0.141030,2.239912,4.314906,3.839279,3.739505,3.208159,...,-0.056207,1.033108,0.375648,-4.418900,-11.233198,-9.623874,-7.846038,-3.918458,11.359954,2
1981-03-31,-0.682913,13.016933,18.200739,10.105100,8.818761,6.152283,1.618636,-0.760436,-1.152443,-0.847500,...,-4.575551,-3.137307,-0.836810,-1.063815,5.814612,17.894457,28.788661,37.550796,39.694009,2
1981-03-31,-2.129921,0.814658,-8.205148,-10.292154,-7.882258,-6.690660,-6.864462,-6.210511,-5.382856,-4.507574,...,0.188999,2.644482,5.454335,6.813375,4.102965,6.380669,7.398740,7.676628,19.064614,2


### calculates the ensemble mean 

In [18]:
data = data.groupby('instance').mean()

In [19]:
data.head()

,"(-70.0, 70.0)","(-70.0, 72.5)","(-70.0, 75.0)","(-70.0, 77.5)","(-70.0, 80.0)","(-70.0, 82.5)","(-70.0, 85.0)","(-70.0, 87.5)","(-70.0, 90.0)","(-70.0, 92.5)",...,"(60.0, 280.0)","(60.0, 282.5)","(60.0, 285.0)","(60.0, 287.5)","(60.0, 290.0)","(60.0, 292.5)","(60.0, 295.0)","(60.0, 297.5)","(60.0, 300.0)",cat3
instance,,,,,,,,,,,,,,,,,,,,,
1981-03-31,-2.587615,-0.549062,-0.506165,-1.069301,-0.995653,-0.993380,-0.983299,-0.916430,-0.752545,-0.663809,...,0.312658,0.471073,0.501731,-0.925579,-0.383086,0.669750,1.729138,0.622537,9.583492,2
1981-04-30,5.434813,5.458168,6.687006,4.068914,2.707874,2.356007,2.382851,2.117046,1.755144,1.458477,...,-2.586753,-1.090671,0.220050,0.988426,3.206794,4.488215,3.606708,1.342927,3.169259,2
1981-05-31,3.272371,-0.418349,-1.748034,-0.684682,-0.557641,-0.363325,-0.166805,0.039878,0.004237,-0.085201,...,2.317343,3.388092,2.212402,0.635666,-0.773588,-1.184573,-1.781298,-1.255870,-5.359990,2
1981-06-30,1.154689,0.840671,0.777285,0.294217,-0.499964,-0.358135,-0.245838,-0.544037,-0.857664,-0.898803,...,-5.974542,-5.648559,-4.763154,-2.560571,1.580788,4.223932,2.987228,6.236104,4.017026,3
1981-07-31,4.031601,1.850137,2.148443,1.542501,0.303050,0.261672,0.478606,0.332814,0.274453,0.354481,...,-3.208066,-4.994429,-3.692417,-3.317991,-3.480191,-2.661617,-3.671462,-2.340320,0.045520,2


In [20]:
data.tail()

,"(-70.0, 70.0)","(-70.0, 72.5)","(-70.0, 75.0)","(-70.0, 77.5)","(-70.0, 80.0)","(-70.0, 82.5)","(-70.0, 85.0)","(-70.0, 87.5)","(-70.0, 90.0)","(-70.0, 92.5)",...,"(60.0, 280.0)","(60.0, 282.5)","(60.0, 285.0)","(60.0, 287.5)","(60.0, 290.0)","(60.0, 292.5)","(60.0, 295.0)","(60.0, 297.5)","(60.0, 300.0)",cat3
instance,,,,,,,,,,,,,,,,,,,,,
2019-08-31,-0.608897,0.840550,2.187259,1.592793,0.684570,0.303771,0.273542,0.121386,-0.042299,-0.261851,...,0.912996,0.528157,0.316847,2.077316,4.152929,1.604906,-1.731637,-0.927909,-4.291195,2
2019-09-30,1.624147,-0.348583,-0.194622,0.913234,0.317423,0.428377,1.192421,1.375572,1.251173,1.314091,...,5.552839,-0.708888,-4.139089,1.770182,1.507730,-0.074339,2.158722,5.229980,2.571010,2
2019-10-31,6.832069,5.497627,7.364498,4.040348,2.414827,3.346765,4.629980,4.673066,4.188982,3.886292,...,0.497016,0.459985,-1.454169,-0.445378,5.049006,3.743771,3.622915,4.021067,3.672556,3
2019-11-30,-4.175055,-3.305227,-4.483405,-1.534297,-0.867517,-1.096882,-1.395082,-1.330199,-1.323345,-1.454840,...,0.918505,-0.103136,2.441998,0.394037,0.358405,1.641052,8.386858,7.271169,11.022463,3
2019-12-31,-1.762336,-0.614903,-0.565931,-0.486907,-0.081162,-0.169381,-0.462178,-0.580092,-0.556294,-0.468798,...,0.884452,-0.275588,-0.420075,-0.723772,-3.157579,-0.805484,1.171506,-2.941425,-1.540671,3


In [21]:
data.shape

(466, 4930)

In [22]:
train_data = data.loc[None:'2015-12',:]

In [24]:
test_data = data.loc['2016-01':None, :]

### saved models 

In [25]:
saved_models = pathlib.Path('./saved_models/AUTOGLUON_v3/')

In [26]:
opath = saved_models.joinpath(f'./autogluon_exp_SKPCA_{GCM}_1981_2010_pred_{application}_reg_{varname}_targetvar_{target_type}_target_type_{var_X}')

In [27]:
if not opath.exists(): 
    opath.mkdir(parents=True)

### initialise repeated stratified cross-validation

In [28]:
kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=2).split(train_data.drop(labels=[target_type],axis=1).values, train_data.loc[:,target_type].values)

In [29]:
percent_variance = 0.8

In [ ]:
test_indices = []
y_preds = []
leader_board = []
perfs = []

for k, (train, test) in enumerate(kfold):
    
    print(f"ENTERING FOLD {k} ---- ")
    
    # saves the test indices
    test_indices.append(test)
    
    # get the numpy array containing the training set initial features (grid points)
    X_train = train_data.drop(labels=[target_type],axis=1).values[train]
    
    # get the numpy array containing the training set target values (y)
    y_train =  train_data.loc[:,target_type].values[train]
    
    # get the numpy array containing the test set initial features (grid points)
    X_test = train_data.drop(labels=[target_type],axis=1).values[test]
    
    # get the numpy array containing the test set target values (y)
    y_test = train_data.loc[:,target_type].values[test]
    
    # -----------
    # standardize 
    
    # initialise the scaler (standard scaler)
    scaler = StandardScaler() 
    
    # fit on the training set features array, and transform to obtain standardized values
    X_train_std = scaler.fit_transform(X_train)
    
    # apply the transformation on the test set initial features 
    X_test_std = scaler.transform(X_test)
    
    # -----------------------------
    # Principal Component Analysis 
    
    # instantiate the pca class, with percent of variance to keep  
    
    skpca = PCA(n_components=percent_variance)
    
    # fit on the training initial (standardized) fedatures array, and transform to obtain the PCs
    X_train_PC = skpca.fit_transform(X_train_std)
    
    # apply the transformation on the test set standardized features 
    X_test_PC = skpca.transform(X_test_std)
    
    # assign the training set PCs to a DataFrame 
    df_train = pd.DataFrame(X_train_PC) 
    
    # add the target values to the training DataFrame 
    df_train.loc[:,target_type] = y_train
    
    # assign the test set Pcs to a DataFrame 
    df_test = pd.DataFrame(X_test_PC) 
    
    # add the target values to the test DataFrame
    df_test.loc[:,target_type] = y_test
    
    # fit the task predictor on the training set DataFrame 
    predictor = task.fit(train_data=df_train, label=target_type, auto_stack=True, output_directory=opath)
    
    # predict the probabilities for each class from the test set features DataFrame (droping the target values column)
#     y_pred_proba = predictor.predict_proba(df_test.drop(labels=[region_name],axis=1))
    
    # predict the class value itself
    y_pred = predictor.predict(df_test.drop(labels=[target_type],axis=1))
    
    # records the probabilities for the classes on the test set 
    y_preds.append(y_pred)
    
    # get the leaderboard DataFrame 
    d = predictor.leaderboard(silent=True)
    
    # records the leaderboard DataFrame 
    leader_board.append(d)
    
    perfs.append(predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True, silent=True))
    
    print(f"EXITING FOLD {k} ---- ")

ENTERING FOLD 0 ---- 


Beginning AutoGluon training ...
AutoGluon will save models to saved_models/AUTOGLUON_v3/autogluon_exp_SKPCA_ECMWF_1981_2010_pred_Apple_and_Pears_reg_RAIN_BC_targetvar_cat3_target_type_PRECIP/
AutoGluon Version:  0.0.12
Train Data Rows:    334
Train Data Columns: 74
Preprocessing data ...
Here are the 3 unique label values in your data:  [2, 3, 1]
AutoGluon infers your prediction problem is: multiclass  (because dtype of label-column == int, but few unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Train Data Class Count: 3
Feature Generator processed 334 data points with 73 features
Original Features (raw dtypes):
	float64 features: 73
Original Features (inferred dtypes):
	float features: 73
Generated Features (special dtypes):
Final Features (raw dtypes):
	float features: 73
Final Features:
	float features: 73
	Data preprocessing and feature enginee

EXITING FOLD 0 ---- 
ENTERING FOLD 1 ---- 


Beginning AutoGluon training ...
AutoGluon will save models to saved_models/AUTOGLUON_v3/autogluon_exp_SKPCA_ECMWF_1981_2010_pred_Apple_and_Pears_reg_RAIN_BC_targetvar_cat3_target_type_PRECIP/
AutoGluon Version:  0.0.12
Train Data Rows:    334
Train Data Columns: 73
Preprocessing data ...
Here are the 3 unique label values in your data:  [2, 3, 1]
AutoGluon infers your prediction problem is: multiclass  (because dtype of label-column == int, but few unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Train Data Class Count: 3
Feature Generator processed 334 data points with 72 features
Original Features (raw dtypes):
	float64 features: 72
Original Features (inferred dtypes):
	float features: 72
Generated Features (special dtypes):
Final Features (raw dtypes):
	float features: 72
Final Features:
	float features: 72
	Data preprocessing and feature enginee

EXITING FOLD 1 ---- 
ENTERING FOLD 2 ---- 


Beginning AutoGluon training ...
AutoGluon will save models to saved_models/AUTOGLUON_v3/autogluon_exp_SKPCA_ECMWF_1981_2010_pred_Apple_and_Pears_reg_RAIN_BC_targetvar_cat3_target_type_PRECIP/
AutoGluon Version:  0.0.12
Train Data Rows:    334
Train Data Columns: 75
Preprocessing data ...
Here are the 3 unique label values in your data:  [2, 3, 1]
AutoGluon infers your prediction problem is: multiclass  (because dtype of label-column == int, but few unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Train Data Class Count: 3
Feature Generator processed 334 data points with 74 features
Original Features (raw dtypes):
	float64 features: 74
Original Features (inferred dtypes):
	float features: 74
Generated Features (special dtypes):
Final Features (raw dtypes):
	float features: 74
Final Features:
	float features: 74
	Data preprocessing and feature enginee

EXITING FOLD 2 ---- 
ENTERING FOLD 3 ---- 


Beginning AutoGluon training ...
AutoGluon will save models to saved_models/AUTOGLUON_v3/autogluon_exp_SKPCA_ECMWF_1981_2010_pred_Apple_and_Pears_reg_RAIN_BC_targetvar_cat3_target_type_PRECIP/
AutoGluon Version:  0.0.12
Train Data Rows:    335
Train Data Columns: 74
Preprocessing data ...
Here are the 3 unique label values in your data:  [2, 3, 1]
AutoGluon infers your prediction problem is: multiclass  (because dtype of label-column == int, but few unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Train Data Class Count: 3
Feature Generator processed 335 data points with 73 features
Original Features (raw dtypes):
	float64 features: 73
Original Features (inferred dtypes):
	float features: 73
Generated Features (special dtypes):
Final Features (raw dtypes):
	float features: 73
Final Features:
	float features: 73
	Data preprocessing and feature enginee

In [ ]:
!pwd